In [1]:
# importing dependencies
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

from flask import Flask, jsonify

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model 
base = automap_base()

# reflect the tables
base.prepare(engine, reflect=True)

# View all of the classes that automap found
base.classes.keys()

# Save references to each table
measurement = base.classes.measurement
station = base.classes.measurement

# Create App
app = Flask(__name__)

In [ ]:
# Static Routes
@app.route("/")
def home():
    return """
    <h1>Welcome to the Hawaii Climate Analysis API!</h1><br/>
    <h2>Available Routes:</h2><br/>
        /api/v1.0/precipitation<br/>
        /api/v1.0/stations<br/>
        /api/v1.0/tobs<br/>
        /api/v1.0/<start></br>
        api/v1.0/<start>/<end></br>
    """

# Precipitation
@app.route("/api/v1.0/precipitation")
def precipitation():

    #link to DB
    session = Session(engine)

    # Calculate the date one year from the last date in data set.
    target_date = dt.date(2017, 8, 23)
    delta = dt.timedelta(days=365)
    query_date = target_date - delta

    # Perform a query to retrieve the data and precipitation scores
    results = session.query(measurement.date, measurement.prcp).\
        filter(measurement.date >= query_date).order_by(measurement.date.desc()).all()
    session.close()

    # Create a dictionary from the row data and append to a list of all results
    precipitation_list = []
    for i in range(len(results)):
        precipitation_dict = {}
        precipitation_dict[results[i][0]] = results[i][1]
        precipitation_list.append(precipitation_dict)

    return jsonify(precipitation_list)

# Stations
@app.route("/api/v1.0/stations")
def station():

    #link to DB
    session = Session(engine)

    # query for station names
    results = session.query(station.name).all()
    session.close()

    # Create a dictionary from the row data and append to a list of all results
    station_data = []
    for station in results:
        station_dict = {}
        station_dict["Station"] = station.station
        station_dict["Name"] = station.name
        station_data.append(station_dict)

    return jsonify(station_data)


# Tobs
@app.route("/api/v1.0/tobs")
def tobs():
    
    #link to DB
    session = Session(engine)
    
    # Calculate the date one year from the last date in data set.
    target_date = dt.date(2017, 8, 23)
    delta = dt.timedelta(days=365)
    query_date = target_date - delta

    # Perform a query to retrieve the data for the most acive station (USC00519281)
    results = session.query(measurement.date, measurement.tobs).\
        filter(measurement.station == 'USC00519281').\
        filter(measurement.date >= query_date).all()
    session.close()

    # create a dictionary of temperatures for the station
    temp_data = []
    for day in results:
        temp_dict = {}
        temp_dict[day.date] = day.tobs
        temp_data.append(temp_dict)

    return jsonify(temp_dict)
       

#start route - Flask
@app.route("/api/v1.0/<start>")
def temperature_s(start):
   
    #link to DB
    session = Session(engine)
   
    # Set start and end dates for date range
    start_date = dt.datetime.strptime(start, "%Y-%m-%d")
    end_date = dt.datetime.strptime("2017-08-23", "%Y-%m-%d")

    # Date range
    # Getting date range
    delta = end_date - start_date
    date_range = []
    for i in range(delta.days + 1):
        date_range.append(start_date + delta(days=i))
    
    # Converting to strings to filter
    str_date_range = []
    for date in date_range:
        new_date = date.strptime("%Y-%m-%d")
        str_date_range.append(new_date)

    # Grabbing avg, min & max temps    
    temp_avg = session.query(func.avg(measurement.tobs))\
                .filter(measurement.date.in_(str_date_range))[0][0]
    temp_min = session.query(func.min(measurement.tobs))\
                .filter(measurement.date.in_(str_date_range))[0][0]
    temp_max = session.query(func.max(measurement.tobs))\
                .filter(measurement.date.in_(str_date_range))[0][0]

    # Dictionary of temperatures
    temp_dict = {}
    temp_dict["Average Temperature"] = temp_avg
    temp_dict["Minimum Temperature"] = temp_min
    temp_dict["Maximum Temperature"] = temp_max

    return jsonify(temp_dict)


#start-end - Flask
@app.route("/api/v1.0/<start>/<end>")
def temperature(start, end):

    #link to DB
    session = Session(engine)

    # Set start and end dates for date range
    start_date = dt.datetime.strptime(start, "%Y-%m-%d")
    end_date = dt.datetime.strptime(end, "%Y-%m-%d")

    # Date range
    # Getting date range
    delta = end_date - start_date
    date_range = []
    for i in range(delta.days + 1):
        date_range.append(start_date + delta(days=i))
    
    # Converting to strings to filter
    str_date_range = []
    for date in date_range:
        new_date = date.strftime("%Y-%m-%d")
        str_date_range.append(new_date)

    # Grabbing avg, min & max temps    
    temp_avg = session.query(func.avg(measurement.tobs))\
                .filter(measurement.date.in_(str_date_range))[0][0]
    temp_min = session.query(func.min(measurement.tobs))\
                .filter(measurement.date.in_(str_date_range))[0][0]
    temp_max = session.query(func.max(measurement.tobs))\
                .filter(measurement.date.in_(str_date_range))[0][0]

    # Dictionary of temperatures
    temp_dict = {}
    temp_dict["Average Temperature"] = temp_avg
    temp_dict["Minimum Temperature"] = temp_min
    temp_dict["Maximum Temperature"] = temp_max

    return jsonify(temp_dict)





# Run the Flask Application
if __name__ == '__main__':
    app.run(debug=True)